In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
df = pd.read_csv('data/MBTI500.csv')
df['posts'] = df['posts'].apply(lambda x: x.split(' '))

In [3]:
print(df.type.value_counts())
lengths = df['posts'].apply(lambda x: len(x))
lengths.describe()

type
INTP    24961
INTJ    22427
INFJ    14963
INFP    12134
ENTP    11725
ENFP     6167
ISTP     3424
ENTJ     2955
ESTP     1986
ENFJ     1534
ISTJ     1243
ISFP      875
ISFJ      650
ESTJ      482
ESFP      360
ESFJ      181
Name: count, dtype: int64


count    106067.000000
mean        500.019818
std           1.209997
min         479.000000
25%         500.000000
50%         500.000000
75%         500.000000
max         527.000000
Name: posts, dtype: float64

In [4]:
from sklearn.utils import resample

df['posts'] = df['posts'].apply(lambda x: x[:479])

INTP = resample(df[df.type == 'INTP'], replace=False, n_samples=181, random_state=4)
INTJ = resample(df[df.type == 'INTJ'], replace=False, n_samples=181, random_state=4)
INFJ = resample(df[df.type == 'INFJ'], replace=False, n_samples=181, random_state=4)
INFP = resample(df[df.type == 'INFP'], replace=False, n_samples=181, random_state=4)
ENTP = resample(df[df.type == 'ENTP'], replace=False, n_samples=181, random_state=4)
ENFP = resample(df[df.type == 'ENFP'], replace=False, n_samples=181, random_state=4)
ISTP = resample(df[df.type == 'ISTP'], replace=False, n_samples=181, random_state=4)
ENTJ = resample(df[df.type == 'ENTJ'], replace=False, n_samples=181, random_state=4)
ESTP = resample(df[df.type == 'ESTP'], replace=False, n_samples=181, random_state=4)
ENFJ = resample(df[df.type == 'ENFJ'], replace=False, n_samples=181, random_state=4)
ISTJ = resample(df[df.type == 'ISTJ'], replace=False, n_samples=181, random_state=4)
ISFP = resample(df[df.type == 'ISFP'], replace=False, n_samples=181, random_state=4)
ISFJ = resample(df[df.type == 'ISFJ'], replace=False, n_samples=181, random_state=4)
ESTJ = resample(df[df.type == 'ESTJ'], replace=False, n_samples=181, random_state=4)
ESFP = resample(df[df.type == 'ESFP'], replace=False, n_samples=181, random_state=4)
ESFJ = df[df.type == 'ESFJ']

uniform_df = pd.concat([INTP, INTJ, INFJ, INFP, ENTP, ENFP, ISTP, ENTJ, ESTP, ENFJ, ISTJ, ISFP, ISFJ, ESTJ, ESFP, ESFJ])

In [5]:
print(uniform_df.type.value_counts())
lengths = uniform_df['posts'].apply(lambda x: len(x))
lengths.describe()

type
INTP    181
INTJ    181
INFJ    181
INFP    181
ENTP    181
ENFP    181
ISTP    181
ENTJ    181
ESTP    181
ENFJ    181
ISTJ    181
ISFP    181
ISFJ    181
ESTJ    181
ESFP    181
ESFJ    181
Name: count, dtype: int64


count    2896.0
mean      479.0
std         0.0
min       479.0
25%       479.0
50%       479.0
75%       479.0
max       479.0
Name: posts, dtype: float64

In [8]:
uniform_df.to_csv('data/UniformMBTI.csv', index=False)

In [6]:
from sklearn.model_selection import train_test_split

X = np.array(uniform_df['posts'])
Y = np.array(uniform_df['type'])

X_train, X_test_val, Y_train, Y_test_val = train_test_split(X, Y, test_size=0.3, random_state=4)
X_test, X_val = X_test_val[:int(len(X_test_val) * 0.5)], X_test_val[int(len(X_test_val) * 0.5):]
Y_test, Y_val = Y_test_val[:int(len(Y_test_val) * 0.5)], Y_test_val[int(len(Y_test_val) * 0.5):]
